# Traffic Sign Recognition Model
## Environment Initialisation

In [1]:
def IntialiseEnv():
    nb_dir = os.path.split(os.getcwd())[0]
    if nb_dir not in sys.path:
        sys.path.append(nb_dir)

In [2]:
from google.colab import drive
drive.mount('/content/drive')
import os
os.chdir("/content/drive/")
!ls
import os
os.chdir("My Drive/[ Masters ] - Deep Learning Proj/Model")
!pip install import_ipynb

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'My Drive'


In [3]:
# Libraries
import os
import sys
import tensorflow as tf
import numpy as np
import math
IntialiseEnv()
import import_ipynb
import pickle
from sklearn.model_selection import train_test_split

# from Data_Preparation.Data_Preparation_Belgium import LoadBelgiumTrainDataSet
# from Data_Preparation.Data_Preparation_Belgium import LoadBelgiumTestDataSet

## Use GPU/ CPU Configuration 
Tensorflow version 2.0
Prompt to user if CPU/ GPU is in use with device name

In [4]:
# Use GPU/CPU Configurations

import tensorflow as tf
print(tf.__version__)
print(tf.test.gpu_device_name())
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

2.3.0
/device:GPU:0
Num GPUs Available:  1


## Reading DataSet
Pre-processing and Loading Train and Test DataSet respectively

In [5]:
def ReadTrainDataSet():
    print("Reading Train Pre-processed DataSet")
    processedTrainDataSet = '../DataSet/Processed_DataSet/BelgiumTrainDataSet.pkl'
    isProcessedTrainDataSetExits= os.path.exists(processedTrainDataSet)
    train_image_array, train_image_labels = None, None
    
    if isProcessedTrainDataSetExits:
        print("Loading Processed Train DataSet from Processed_DataSet/BelgiumTrainDataSet.pkl")
        file = open(processedTrainDataSet, 'rb')
        train_image_array, train_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Train DataSet.")
        
    else:
        print("Processed_DataSet/BelgiumTrainDataSet.pkl file does not exist")
        print("Loading Train DataSet ... This may take a while.")
        train_image_array, train_image_labels =  LoadBelgiumTrainDataSet()
        file = open(processedTrainDataSet, 'wb')
        pickle.dump((train_image_array, train_image_labels), file, protocol=4)
        file.close()
        print("Saving pre-processed train DataSets in Processed_DataSet/BelgiumTrainDataSet.pkl")
    return train_image_array, train_image_labels    

In [6]:
def ReadTestDataSet():
    processedTestDataSet = '../DataSet/Processed_DataSet/BelgiumTestDataSet.pkl'
    isProcessedTestDataSetExits= os.path.exists(processedTestDataSet)
    test_image_array, test_image_labels = None, None
    
    if isProcessedTestDataSetExits:
        print("Loading Processed Test DataSet from Processed_DataSet/BelgiumTestDataSet.pkl")
        file = open(processedTestDataSet, 'rb')
        test_image_array, test_image_labels = pickle.load(file)
        file.close()
        print("Done Loading Test DataSet.")
    else:
        print("Processed_DataSet/BelgiumTestDataSet.pkl file does not exist")
        print("Loading Test DataSet ... This may take a while.")
        test_image_array, test_image_labels = LoadBelgiumTestDataSet()
        file = open(processedTestDataSet, 'wb')
        pickle.dump((test_image_array, test_image_labels), file)
        file.close()
        print("Saving pre-processed test DataSets in Processed_DataSet/BelgiumTestDataSet.pkl")
    return test_image_array, test_image_labels

In [7]:
train_image_array, train_image_labels= ReadTrainDataSet()

Reading Train Pre-processed DataSet
Loading Processed Train DataSet from Processed_DataSet/BelgiumTrainDataSet.pkl
Done Loading Train DataSet.


In [8]:
print(train_image_array.shape)

(4575, 60, 60, 1)


In [9]:
test_image_array, test_image_labels= ReadTestDataSet()

Loading Processed Test DataSet from Processed_DataSet/BelgiumTestDataSet.pkl
Done Loading Test DataSet.


In [10]:
print(test_image_array.shape)

(2520, 60, 60, 1)


## Train-Split 
Split Train images dataset into two splits: training and validation respectively 90-10

In [11]:
image_train, image_valid,label_train, label_valid = train_test_split(train_image_array, train_image_labels,stratify=train_image_labels,test_size=0.1,random_state=43)

In [12]:
print(image_train.shape)

(4117, 60, 60, 1)


In [13]:
del train_image_array
del train_image_labels

## Hyper-parameters Tuning

In [14]:
# Hyper-parameters tuning
kernel_2 = (3,3)
pooling = (2,2)
dropout = 0.2
num_classes = 62

## Model Architecture

In [15]:
import tensorflow as tf
import keras

from keras.layers import Dense, Dropout, Activation, Flatten, BatchNormalization
from keras.layers import Conv2D, MaxPool2D, Add,AveragePooling2D
from keras.models import Sequential
from keras.layers import Conv2D, SeparableConv2D,BatchNormalization,Dropout,MaxPool2D,Flatten,Dense
from keras.layers import concatenate

from keras import initializers
from keras import backend as K
from keras.layers import Input
from keras.models import Model

In [16]:
weight_decay=1E-4
from keras.regularizers import l2,l1_l2

def model_():
    model = None
    tf.initializers.Orthogonal(gain=1.0, seed=None)
    model=Sequential()

    model.add(Conv2D(16,kernel_2, input_shape=(60,60,1), strides = 1, padding='valid',activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(32,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    model.add(Conv2D(64,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.01)))
    model.add(BatchNormalization())
    model.add(Conv2D(128,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())

    model.add(Conv2D(256,kernel_2, strides = 1, padding='valid', activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(BatchNormalization())
    model.add(Conv2D(256,kernel_2, activation = tf.nn.relu,kernel_regularizer=l2(0.1)))
    model.add(MaxPool2D(pooling))
    model.add(Dropout(dropout))
    model.add(BatchNormalization())
    model.add(Flatten())
    model.add(Dense(512, activation = tf.nn.relu))
    model.add(Dropout(0.4))
    model.add(Dense(num_classes, activation = 'softmax'))
    return model

In [17]:
from tensorflow import keras
model = model_()

## Model Optimizer 

In [18]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', factor=0.2,
                              patience=3,  verbose=1, min_delta=1e-3, min_lr=1e-20)

model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

In [19]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
aug = ImageDataGenerator(featurewise_center=False, 
                            featurewise_std_normalization=False, 
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.2,
                            shear_range=0.1,
                            rotation_range=10.,)

In [20]:
x=aug.flow(image_train, np.array(label_train), batch_size=32)

## Model Training

In [21]:
history = model.fit(x, epochs=95, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/95
129/129 [==============================] - 3s 22ms/step - loss: 43.8324 - sparse_categorical_accuracy: 0.3658 - val_loss: 35.6173 - val_sparse_categorical_accuracy: 0.0284
Epoch 2/95
129/129 [==============================] - 2s 19ms/step - loss: 26.4801 - sparse_categorical_accuracy: 0.6216 - val_loss: 23.3056 - val_sparse_categorical_accuracy: 0.1201
Epoch 3/95
129/129 [==============================] - 2s 19ms/step - loss: 16.4498 - sparse_categorical_accuracy: 0.7010 - val_loss: 14.1793 - val_sparse_categorical_accuracy: 0.3930
Epoch 4/95
129/129 [==============================] - 2s 19ms/step - loss: 10.4455 - sparse_categorical_accuracy: 0.7527 - val_loss: 8.5329 - val_sparse_categorical_accuracy: 0.6900
Epoch 5/95
129/129 [==============================] - 2s 19ms/step - loss: 6.8399 - sparse_categorical_accuracy: 0.7921 - val_loss: 5.5393 - val_sparse_categorical_accuracy: 0.8188
Epoch 6/95
129/129 [==============================] - 2s 19ms/step - loss: 4.6787 - spar

In [24]:
history2 = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
129/129 [==============================] - 3s 19ms/step - loss: 0.1344 - sparse_categorical_accuracy: 0.9930 - val_loss: 0.1119 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1350 - sparse_categorical_accuracy: 0.9932 - val_loss: 0.1102 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1344 - sparse_categorical_accuracy: 0.9927 - val_loss: 0.1091 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1278 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.1073 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1344 - sparse_categorical_accuracy: 0.9913 - val_loss: 0.1068 - val_sparse_categorical_accuracy: 1.0000


In [26]:
history3 = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1258 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.1077 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1286 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.1069 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/5
129/129 [==============================] - 3s 19ms/step - loss: 0.1237 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.1044 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1235 - sparse_categorical_accuracy: 0.9942 - val_loss: 0.1044 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/5
129/129 [==============================] - 2s 19ms/step - loss: 0.1190 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.1044 - val_sparse_categorical_accuracy: 1.0000


In [28]:
history4 = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1256 - sparse_categorical_accuracy: 0.9932 - val_loss: 0.1027 - val_sparse_categorical_accuracy: 1.0000
Epoch 2/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1209 - sparse_categorical_accuracy: 0.9951 - val_loss: 0.1014 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1220 - sparse_categorical_accuracy: 0.9942 - val_loss: 0.1001 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1180 - sparse_categorical_accuracy: 0.9961 - val_loss: 0.0990 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1159 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.1008 - val_sparse_categorical_accuracy: 1.0000


In [30]:
history5 = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1142 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.1011 - val_sparse_categorical_accuracy: 0.9956
Epoch 2/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1157 - sparse_categorical_accuracy: 0.9947 - val_loss: 0.0978 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1161 - sparse_categorical_accuracy: 0.9939 - val_loss: 0.0968 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1120 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.1014 - val_sparse_categorical_accuracy: 0.9956
Epoch 5/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1111 - sparse_categorical_accuracy: 0.9964 - val_loss: 0.0948 - val_sparse_categorical_accuracy: 1.0000


In [32]:
history6 = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1115 - sparse_categorical_accuracy: 0.9949 - val_loss: 0.0971 - val_sparse_categorical_accuracy: 0.9956
Epoch 2/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1102 - sparse_categorical_accuracy: 0.9954 - val_loss: 0.0934 - val_sparse_categorical_accuracy: 1.0000
Epoch 3/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1130 - sparse_categorical_accuracy: 0.9937 - val_loss: 0.0951 - val_sparse_categorical_accuracy: 0.9978
Epoch 4/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1092 - sparse_categorical_accuracy: 0.9959 - val_loss: 0.0928 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/5
129/129 [==============================] - 3s 19ms/step - loss: 0.1061 - sparse_categorical_accuracy: 0.9964 - val_loss: 0.0960 - val_sparse_categorical_accuracy: 0.9956


In [34]:
history7 = model.fit(x, epochs=5, shuffle=True, validation_data=(image_valid, np.array(label_valid)),verbose=1,callbacks=[reduce_lr])             

Epoch 1/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1096 - sparse_categorical_accuracy: 0.9949 - val_loss: 0.0952 - val_sparse_categorical_accuracy: 0.9956
Epoch 2/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1085 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.0933 - val_sparse_categorical_accuracy: 0.9978
Epoch 3/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1088 - sparse_categorical_accuracy: 0.9944 - val_loss: 0.0914 - val_sparse_categorical_accuracy: 1.0000
Epoch 4/5
129/129 [==============================] - 3s 20ms/step - loss: 0.1061 - sparse_categorical_accuracy: 0.9956 - val_loss: 0.0908 - val_sparse_categorical_accuracy: 1.0000
Epoch 5/5
129/129 [==============================] - 3s 19ms/step - loss: 0.1052 - sparse_categorical_accuracy: 0.9959 - val_loss: 0.0891 - val_sparse_categorical_accuracy: 1.0000


## Model Evaluation

In [40]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 21ms/step - loss: 0.1578 - sparse_categorical_accuracy: 0.9925


[0.15781962871551514, 0.992460310459137]

In [35]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 17ms/step - loss: 0.1578 - sparse_categorical_accuracy: 0.9925


[0.15781962871551514, 0.992460310459137]

In [33]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 17ms/step - loss: 0.1648 - sparse_categorical_accuracy: 0.9881


[0.16479891538619995, 0.988095223903656]

In [31]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 21ms/step - loss: 0.1676 - sparse_categorical_accuracy: 0.9905


[0.16764642298221588, 0.9904761910438538]

In [29]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 21ms/step - loss: 0.1727 - sparse_categorical_accuracy: 0.9913


[0.17268019914627075, 0.9912698268890381]

In [27]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 20ms/step - loss: 0.1731 - sparse_categorical_accuracy: 0.9913


[0.17313072085380554, 0.9912698268890381]

In [25]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 20ms/step - loss: 0.1864 - sparse_categorical_accuracy: 0.9893


[0.186386376619339, 0.9892857074737549]

In [22]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

10/10 [==============================] - 0s 36ms/step - loss: 0.1839 - sparse_categorical_accuracy: 0.9909


[0.18386822938919067, 0.9908730387687683]

In [23]:
xxxx

NameError: ignored

In [ ]:
model.evaluate(test_image_array, np.array(test_image_labels), batch_size=256)

## Visualization

In [ ]:
import matplotlib.pyplot as plt
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

## Model Summary 
Number of parameters used in the model architecture

In [ ]:
print(model.summary())

## Saving Trained .h5 Model

In [36]:
def saveCNNModel(model_name):
    model.save(model_name)

In [39]:
saveCNNModel('Trained_Models/Belgium_Winning_99_25_.h5')

In [ ]:
model._set_inputs(image_train,np.array(label_train))

## Loading Saved Trained .h5 Model

In [53]:
del model

In [54]:
def load_model():
    Model = tf.keras.models.load_model('Trained_Models/Belgium_Winning_99_25_.h5')
    return Model

In [55]:
model= load_model()

In [ ]:
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])

## Computation Avg. Processing Time

In [ ]:
import time
t1=time.time()
model.predict(test_image_array)
t2=time.time()
print("Average Processing time: ", ((t2-t1)/12630)*1000, " ms")